# Create ISA-API Investigation from Datascriptor Study Design configuration

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 6-arm study design. Each arm has 10 subjects
- there is an observational factor with 3 values, which is age_group (young, middle-aged, elderly)
- a crossover of two treatments, a drug treatment and a biological treatment
- three non-treatment phases: screen, washout and follow-up
- two sample types colllected: blood and saliva
- two assay types: 
    - metabolite profiling through mass spectrometry on the saliva sample. The mass spec will be run on a "Agilent QTQF 6510" instrument, testing both "FIA" and "LC" injection modes, and "positive" acquisition mode.
    - metabolite profiling through  NMR spectroscopy on the blood samples.  The NMR will be run on a "Bruker Avance II 1 GHz" instrument, on "1D 1H NMR" acquisition mode, testing both "CPGM" amd "TOCSY" pulse sequences.

## 1. Setup

Let's import all the required libraries

In [1]:
# If executing the notebooks on `Google Colab`,uncomment the following command 
# and run it to install the required python libraries. Also, make the test datasets available.

# !pip install -r requirements.txt

In [2]:
from time import time
import os
import json

In [3]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [4]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [5]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [6]:
with open(os.path.abspath(os.path.join(
    "isa-study-design-as-json/datascriptor", "crossover-study-design-4-arms-blood-derma-nmr-ms-chipseq.json")), "r") as config_file:
    study_design_config = json.load(config_file)
    
study_design_config

{'name': 'Chained protocols study',
 'subjectType': 'Homo sapiens',
 'subjectSize': 10,
 'designType': {'term': 'crossover design',
  'id': 'OBI:0500003',
  'iri': 'http://purl.obolibrary.org/obo/OBI_0500003',
  'label': 'Study subjects receive repeated treatments',
  'value': 'crossover'},
 'observationalFactors': [{'name': 'sex',
   'values': ['M', 'F'],
   'isQuantitative': False,
   'unit': None},
  {'name': 'condition',
   'values': ['healthy', 'diseased'],
   'isQuantitative': False,
   'unit': None}],
 'subjectGroups': {'selected': [{'name': 'SubjectGroup_0',
    'type': 'Homo sapiens',
    'characteristics': [{'name': 'sex',
      'value': 'M',
      'unit': None,
      'isQuantitative': False},
     {'name': 'condition',
      'value': 'diseased',
      'unit': None,
      'isQuantitative': False}]},
   {'name': 'SubjectGroup_1',
    'type': 'Homo sapiens',
    'characteristics': [{'name': 'sex',
      'value': 'M',
      'unit': None,
      'isQuantitative': False},
     {'na

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design_from_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [7]:
study_design = generate_study_design(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [8]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(studies=[study])

The generation of the study design took 6.24 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [9]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 1.40 s.


In [10]:
directory = os.path.abspath(os.path.join('output'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output','isa-investigation-2-arms-nmr-ms.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [12]:
start = time()
dataframes = isatab.dump_tables_to_dataframes(investigation)
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

# alternatively, if you just want to write the isatab files to file, you can run

# isatab.dump(investigation, os.path.abspath(os.path.join('notebook-output/isa-study-from-design-config')))

In [13]:
len(dataframes)

4

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [14]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT16_protein-DNA-binding-site-identification_nucleic-acid-sequencing.txt'

'a_AT2_metabolite-profiling_NMR-spectroscopy.txt'

'a_AT0_metabolite-profiling_mass-spectrometry.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [15]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP4' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP5' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))
print("There are {} samples in the GRP4 arm (i.e. group)".format(count_arm4_samples))

There are 110 samples in the GRP0 arm (i.e. group)
There are 0 samples in the GRP2 arm (i.e. group)
There are 0 samples in the GRP3 arm (i.e. group)
There are 110 samples in the GRP4 arm (i.e. group)


### 7.1 Overview of the Mass Spec assay table

For the mass. spec. assay table, we have 240 (saliva) samples, 480 extracts (2 per  sample, "lipids" and "polar" fractions), 960 labeled extracts (2 per extract, as "#replicates" is  2) and 3840 mass spec protocols + 3840 output files (4 per labeled extract as we do 2 technical replicates with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [16]:
dataframes['a_AT0_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Performer.1,Labeled Extract Name,Label,Protocol REF.2,Parameter Value[instrument],Parameter Value[injection_mode],Parameter Value[acquisition_mode],MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT0-S6-Extract-R1,non-polar fraction,labeling,Unknown,AT0-S6-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,LC,positive mode,AT0-S6-mass-spectrometry-Acquisition-R2,Unknown,AT0-S6-raw-spectral-data-file-R2.mzML
1,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT0-S5-Extract-R2,polar fraction,labeling,Unknown,AT0-S5-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,FIA,positive mode,AT0-S5-mass-spectrometry-Acquisition-R5,Unknown,AT0-S5-raw-spectral-data-file-R5.mzML
2,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT0-S6-Extract-R2,polar fraction,labeling,Unknown,AT0-S6-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,LC,positive mode,AT0-S6-mass-spectrometry-Acquisition-R7,Unknown,AT0-S6-raw-spectral-data-file-R7.mzML
3,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT0-S6-Extract-R1,non-polar fraction,labeling,Unknown,AT0-S6-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,FIA,positive mode,AT0-S6-mass-spectrometry-Acquisition-R4,Unknown,AT0-S6-raw-spectral-data-file-R4.mzML
4,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT0-S6-Extract-R2,polar fraction,labeling,Unknown,AT0-S6-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,FIA,positive mode,AT0-S6-mass-spectrometry-Acquisition-R6,Unknown,AT0-S6-raw-spectral-data-file-R6.mzML
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,GRP7_SBJ10_A3E5_SMP-blood-6,extraction,Unknown,AT0-S627-Extract-R1,non-polar fraction,labeling,Unknown,AT0-S627-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,FIA,positive mode,AT0-S627-mass-spectrometry-Acquisition-R3,Unknown,AT0-S627-raw-spectral-data-file-R3.mzML
5116,GRP7_SBJ10_A3E5_SMP-blood-6,extraction,Unknown,AT0-S628-Extract-R2,polar fraction,labeling,Unknown,AT0-S628-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,LC,positive mode,AT0-S628-mass-spectrometry-Acquisition-R7,Unknown,AT0-S628-raw-spectral-data-file-R7.mzML
5117,GRP7_SBJ10_A3E5_SMP-blood-6,extraction,Unknown,AT0-S627-Extract-R1,non-polar fraction,labeling,Unknown,AT0-S627-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,LC,positive mode,AT0-S627-mass-spectrometry-Acquisition-R1,Unknown,AT0-S627-raw-spectral-data-file-R1.mzML
5118,GRP7_SBJ10_A3E5_SMP-blood-6,extraction,Unknown,AT0-S627-Extract-R2,polar fraction,labeling,Unknown,AT0-S627-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,FIA,positive mode,AT0-S627-mass-spectrometry-Acquisition-R5,Unknown,AT0-S627-raw-spectral-data-file-R5.mzML


### Overview of the NMR assay table

For the NMR assay table, we have 300 (blood) samples, 1200 extracts (4 per  sample, 2 extraction replicates of the "supernatant" and "pellet" fractions) and 4800 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [17]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[acquisition_mode],Parameter Value[pulse_sequence],Performer.1,Free Induction Decay Data File
0,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,Unknown,AT2-S5-Extract-R2,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,2D 13C-13C NMR,watergate,Unknown,AT2-S5-raw_spectral_data_file-R10.raw
1,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,Unknown,AT2-S5-Extract-R1,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,2D 13C-13C NMR,TOCSY,Unknown,AT2-S5-raw_spectral_data_file-R3.raw
2,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,Unknown,AT2-S5-Extract-R2,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 13C NMR,TOCSY,Unknown,AT2-S5-raw_spectral_data_file-R15.raw
3,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,Unknown,AT2-S5-Extract-R1,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 13C NMR,TOCSY,Unknown,AT2-S5-raw_spectral_data_file-R1.raw
4,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,Unknown,AT2-S5-Extract-R1,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,2D 13C-13C NMR,watergate,Unknown,AT2-S5-raw_spectral_data_file-R8.raw
...,...,...,...,...,...,...,...,...,...,...,...
1275,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,Unknown,AT2-S78-Extract-R1,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,2D 13C-13C NMR,TOCSY,Unknown,AT2-S78-raw_spectral_data_file-R4.raw
1276,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,Unknown,AT2-S78-Extract-R1,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 13C NMR,TOCSY,Unknown,AT2-S78-raw_spectral_data_file-R2.raw
1277,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,Unknown,AT2-S78-Extract-R2,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,2D 13C-13C NMR,TOCSY,Unknown,AT2-S78-raw_spectral_data_file-R11.raw
1278,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,Unknown,AT2-S78-Extract-R2,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 13C NMR,watergate,Unknown,AT2-S78-raw_spectral_data_file-R13.raw


### Overview of the Chip-Seq assay table

For the Chip-Seq assay table, we have 300 (blood) samples, 1200 extracts (4 per  sample, 2 extraction replicates of the "supernatant" and "pellet" fractions).

In [18]:
dataframes['a_AT16_protein-DNA-binding-site-identification_nucleic-acid-sequencing.txt']

,Sample Name,Protocol REF,Parameter Value[cross linking],Parameter Value[DNA fragmentation],Parameter Value[DNA fragment size],Parameter Value[immunoprecipitation antibody],Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[library orientation],Parameter Value[library selection],Parameter Value[library strategy],Performer.1,Protocol REF.2,Parameter Value[sequencing instrument],Assay Name,Performer.2,Raw Data File
0,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,uv-light,nebulization,100 nm,monoclonal,Unknown,AT16-S85-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Ion Torrent S5 XL,AT16-S85-nucleic-acid-sequencing-Acquisition-R3,Unknown,AT16-S85-raw_data_file-R3.raw
1,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,uv-light,nebulization,100 nm,monoclonal,Unknown,AT16-S85-Extract-R1,GENOMIC,library preparation,single-end,PCR,Chip-Seq,Unknown,nucleic acid sequencing,Illumina HiSeq 4000,AT16-S85-nucleic-acid-sequencing-Acquisition-R5,Unknown,AT16-S85-raw_data_file-R5.raw
2,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,formaldehyde,nebulization,100 nm,monoclonal,Unknown,AT16-S5-Extract-R1,GENOMIC,library preparation,single-end,PCR,Chip-Seq,Unknown,nucleic acid sequencing,Ion Torrent S5 XL,AT16-S5-nucleic-acid-sequencing-Acquisition-R5,Unknown,AT16-S5-raw_data_file-R5.raw
3,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,uv-light,nebulization,100 nm,monoclonal,Unknown,AT16-S85-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Ion Torrent S5 XL,AT16-S85-nucleic-acid-sequencing-Acquisition-R1,Unknown,AT16-S85-raw_data_file-R1.raw
4,GRP0_SBJ01_A0E5_SMP-derma-1,extraction,formaldehyde,nebulization,100 nm,monoclonal,Unknown,AT16-S5-Extract-R1,GENOMIC,library preparation,single-end,PCR,Chip-Seq,Unknown,nucleic acid sequencing,Ion Torrent S5 XL,AT16-S5-nucleic-acid-sequencing-Acquisition-R7,Unknown,AT16-S5-raw_data_file-R7.raw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,uv-light,nebulization,100 nm,monoclonal,Unknown,AT16-S158-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Illumina HiSeq 4000,AT16-S158-nucleic-acid-sequencing-Acquisition-R4,Unknown,AT16-S158-raw_data_file-R4.raw
1276,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,formaldehyde,nebulization,100 nm,monoclonal,Unknown,AT16-S78-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Illumina HiSeq 4000,AT16-S78-nucleic-acid-sequencing-Acquisition-R4,Unknown,AT16-S78-raw_data_file-R4.raw
1277,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,formaldehyde,nebulization,100 nm,monoclonal,Unknown,AT16-S78-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Illumina HiSeq 4000,AT16-S78-nucleic-acid-sequencing-Acquisition-R2,Unknown,AT16-S78-raw_data_file-R2.raw
1278,GRP7_SBJ10_A3E5_SMP-derma-2,extraction,uv-light,nebulization,100 nm,monoclonal,Unknown,AT16-S158-Extract-R1,GENOMIC,library preparation,single-end,ChIP,Chip-Seq,Unknown,nucleic acid sequencing,Ion Torrent S5 XL,AT16-S158-nucleic-acid-sequencing-Acquisition-R1,Unknown,AT16-S158-raw_data_file-R1.raw


In [19]:
dataframes['a_AT0_metabolite-profiling_mass-spectrometry.txt'].nunique(axis=0, dropna=True)

Sample Name                           320
Protocol REF                            1
Performer                               1
Extract Name                         1280
Characteristics[extract type]           2
Protocol REF.1                          1
Performer.1                             1
Labeled Extract Name                 1280
Label                                   1
Protocol REF.2                          1
Parameter Value[instrument]             1
Parameter Value[injection_mode]         2
Parameter Value[acquisition_mode]       1
MS Assay Name                        5120
Performer.2                             1
Raw Spectral Data File               5120
dtype: int64

In [20]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt'].nunique(axis=0, dropna=True)

Sample Name                            80
Protocol REF                            1
Performer                               1
Extract Name                          160
Characteristics[extract type]           2
Protocol REF.1                          1
Parameter Value[instrument]             1
Parameter Value[acquisition_mode]       2
Parameter Value[pulse_sequence]         2
Performer.1                             1
Free Induction Decay Data File       1280
dtype: int64

## About this notebook

- authors: philippe.rocca-serra@oerc.ox.ac.uk, massimiliano.izzo@oerc.ox.ac.uk
- license: CC-BY 4.0
- support: isatools@googlegroups.com
- issue tracker: https://github.com/ISA-tools/isa-api/issues